In [1]:
import numpy as np
import pytest
from sklearn.linear_model import LinearRegression, Ridge

## Linear regression

In [13]:
class LinearRegr:
    def fit(self, X, Y):
        # input:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Finds theta minimising quadratic loss function L, using an explicit formula.
        # Note: before applying the formula to X one should append to X a column with ones.
        n, m = X.shape
        X = np.concatenate((np.ones((n, 1), dtype=int), X), axis=1)
        X_T = X.T
        self.theta = np.linalg.inv(X_T @ X) @ X_T @ Y
        #y_pred = X @ self.theta
        #L = sum((Y - y_pred)**2)
        #S = sum((Y - np.mean(Y))**2)
        return self
    
    def predict(self, X):
        
        k, m = X.shape
        if k > 2 or m > 2:
            intercept = self.theta[0]
            coef = self.theta[1:]
            return intercept + X @ coef
        return self.theta[1]*X + self.theta[0]

In [14]:
def test_RegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,2,10]).reshape((3,1))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    assert list(actual) == pytest.approx(list(expected))

def test_RegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    a = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    expected = LinearRegression().fit(X, Y).predict(a)
    actual = LinearRegr().fit(X, Y).predict(a)
    assert list(actual) == pytest.approx(list(expected))

In [15]:
test_RegressionInOneDim()
test_RegressionInThreeDim()

## Ridge regression

In [9]:
ITERATIONS = 10**5
LEARNING_RATE = 10**(-5)

In [10]:
class RidgeRegr:
    def __init__(self, alpha = 0.0):
        self.alpha = alpha

    def basic_fit(self, X, Y):
        # input:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Finds theta (approximately) minimising quadratic loss function L with Ridge penalty,
        # using an iterative method.
        
        
        n, m = X.shape
        X = np.concatenate((np.ones((n, 1), dtype=int), X), axis=1)
        X_T = X.T
        I = np.identity(m+1)
        I[0][0] = 0
        self.theta = np.linalg.inv(X_T @ X + self.alpha*I) @ X_T @ Y
        return self
    
    def ridgeGradient(self, X, Y, predictions):
        grad = -(2/Y.size) * X.T.dot((Y - predictions)) + self.alpha * self.theta
        #grad[0] = - 2 * np.sum(Y - predictions) / Y.size
        return grad
    
    def fit(self, X, Y, num_of_iterations=ITERATIONS, learning_rate=LEARNING_RATE):
        n, m = X.shape
        X = np.concatenate((np.ones((n, 1), dtype=int), X), axis=1)
        X_T = X.T
        self.theta = np.zeros(m+1)
        #self.theta[0] = np.random.normal()
        for i in range(num_of_iterations):
            pred = X @ self.theta
            grad = self.ridgeGradient(X, Y, pred)
            self.theta -= learning_rate*grad 
        print(self.theta)
        return self
    
    def fit2(self, X, Y, num_of_iterations=ITERATIONS, learning_rate=LEARNING_RATE):
        n, m = X.shape
        #X = np.concatenate((np.ones((n, 1), dtype=int), X), axis=1)
        #X_T = X.T
        #self.theta = np.zeros(m+1)
        #self.theta[0] = np.random.normal()
        #for i in range(num_of_iterations):
        #    pred = X @ self.theta
        #    for i in range(len(self.theta)):
        #        if i==0:
        #            derivative = 2 * np.dot(Y-pred, X[:, i])
        #        else:
        #            derivative = 2 * np.dot(Y-pred, X[:, i]) + 2*(self.alpha*self.theta)
        #        #grad = self.ridgeGradient(X, Y, pred)
        #        self.theta -= learning_rate*derivative
        #print(self.theta)
        #return self
        #self.theta = ridge_regression_gradient_descent(X, Y, np.zeros(m), LEARNING_RATE, 1e11, max_iterations=ITERATIONS)
        return self
    
    def predict(self, X):
        # input:
        #  X = np.array, shape = (k, m)
        # returns:
        #  Y = wektor(f(X_1), ..., f(X_k))
        k, m = X.shape
        if k > 2 or m > 2:
            #intercept = self.theta[0]
            #coef = self.theta[1:]
            return  X @ self.theta[1:]
        return self.theta[1]*X# + self.theta[0]

In [11]:
def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    print(Ridge(alpha).fit(X, Y).coef_)
    print(expected)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)

def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    print(Ridge(alpha).fit(X, Y).coef_)
    print(expected)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-3)
    

In [12]:
test_RidgeRegressionInOneDim()
#test_RidgeRegressionInThreeDim()

[1.49171271]
[ 1.88950276  3.38121547 15.31491713]
[0.37289248 1.48681635]
[ 1.48681635  2.9736327  14.86816352]


AssertionError: 